In [1]:
import numpy as np
import string
import time
import os

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

import emotion_model
import dwdii_transforms

Using Theano backend.


In [2]:
imagePath = "/root/facial_expressions/images"
dataPath = "/root/facial_expressions/data/legend.csv"

### Load Training and Test Data

In [3]:
emoMetaData = dwdii_transforms.load_training_metadata(dataPath)

In [4]:
len(emoMetaData)

13682

In [5]:
emoMetaData.keys()[:10]

['King_Abdullah_II_0001.jpg',
 'Manuel_Poggiali_0002.jpg',
 'Paula_Radcliffe_0006.jpg',
 'Jean_Chretien_0048.jpg',
 'Scott_Rolen_0001.jpg',
 'Kate_Capshaw_0001.jpg',
 'David_Howard_0001.jpg',
 'Sheila_Wellstone_0001.jpg',
 'Yannos_Papantoniou_0001.jpg',
 'Kelly_Clarkson_0003.jpg']

In [12]:
maxData = 500
X_data, Y_data = dwdii_transforms.load_data(dataPath, imagePath, maxData = maxData, verboseFreq = 20)
print X_data.shape
print Y_data.shape

0.000000: King_Abdullah_II_0001.jpg
0.040000: Giuseppe_Gibilisco_0003.jpg
0.080000: Don_Lake_0001.jpg
0.120000: Jason_Priestley_0001.jpg
0.160000: Shaukat_Aziz_0002.jpg
0.200000: Jim_Hardin_0001.jpg
0.240000: Ana_Palacio_0003.jpg
0.280000: Nicolas_Escude_0002.jpg
0.320000: Janet_Crawford_0001.jpg
0.360000: Jayamadhuri_59.jpg
0.360000: Taufik_Hidayat_0001.jpg
0.400000: George_W_Bush_0193.jpg
0.440000: Andres_Pastrana_0001.jpg
0.480000: Owen_Wilson_0002.jpg
0.520000: Jackie_Chan_0004.jpg
0.560000: Laura_Bush_0003.jpg
0.600000: Kate_Hudson_0009.jpg
0.640000: Anders_Ebbeson_0001.jpg
0.680000: Milton_Berle_0001.jpg
0.720000: Rudolph_Giuliani_0005.jpg
0.760000: Eugene_Teslovic_0001.jpg
0.800000: Jason_Alexander_0002.jpg
0.840000: Amber_Tamblyn_0001.jpg
0.880000: Yann_Martel_0001.jpg
0.920000: Jeannette_Biedermann_0001.jpg
0.960000: Heather_Locklear_0001.jpg
(500, 350, 350)
(500, 1)


In [13]:
# Split the data into Training and Test sets
trainNdx = int(maxData * .8)
print trainNdx
X_train, X_test = np.split(X_data, [trainNdx])
Y_train, Y_test = np.split(Y_data, [trainNdx])
print X_train.shape
print X_test.shape

print Y_train.shape
print Y_test.shape

400
(400, 350, 350)
(100, 350, 350)
(400, 1)
(100, 1)


### Define the Model

In [14]:
emotions = dwdii_transforms.emotionNumerics()
print emotions
print len(emotions)

{'sadness': 6, 'neutral': 2, 'disgust': 1, 'anger': 0, 'surprise': 4, 'fear': 5, 'happiness': 3}
7


In [15]:
model = emotion_model.emotion_model_v1(len(emotions), verbose=True)

Compiling Model ... 
____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
zeropadding2d_3 (ZeroPadding2D)    (None, 1, 352, 352) 0           zeropadding2d_input_2[0][0]      
____________________________________________________________________________________________________
convolution2d_3 (Convolution2D)    (None, 32, 350, 350)320         zeropadding2d_3[0][0]            
____________________________________________________________________________________________________
maxpooling2d_3 (MaxPooling2D)      (None, 32, 175, 175)0           convolution2d_3[0][0]            
____________________________________________________________________________________________________
zeropadding2d_4 (ZeroPadding2D)    (None, 32, 177, 177)0           maxpooling2d_3[0][0]             
______________________________________________________________________

In [16]:
os.listdir('/root/facial_expressions/data')

['500_picts_satz.csv', 'legend.csv']

In [19]:
testX = X_test.reshape(X_test.shape[0], 1, 350, 350)
trainX = X_train.reshape(X_train.shape[0], 1, 350, 350)

emotion_model.run_network([trainX, testX, Y_train, Y_test], model)

(400, 7)
(100, 7)
Training model...
Train on 400 samples, validate on 100 samples
Epoch 1/20


RuntimeError: BaseCorrMM: Failed to allocate output of 256 x 32 x 350 x 350
Apply node that caused the error: CorrMM{valid, (1, 1)}(IncSubtensor{InplaceSet;::, ::, int64:int64:, int64:int64:}.0, Subtensor{::, ::, ::int64, ::int64}.0)
Toposort index: 59
Inputs types: [TensorType(float32, 4D), TensorType(float32, 4D)]
Inputs shapes: [(256, 1, 352, 352), (32, 1, 3, 3)]
Inputs strides: [(495616, 495616, 1408, 4), (36, 36, -12, -4)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[Elemwise{add,no_inplace}(CorrMM{valid, (1, 1)}.0, Reshape{4}.0), Elemwise{Composite{(i0 * (Abs(i1) + i2 + i3))}}[(0, 2)](TensorConstant{(1, 1, 1, 1) of 0.5}, Elemwise{add,no_inplace}.0, CorrMM{valid, (1, 1)}.0, Reshape{4}.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "<ipython-input-15-eabc56dd355e>", line 1, in <module>
    model = emotion_model.emotion_model_v1(len(emotions), verbose=True)
  File "emotion_model.py", line 30, in emotion_model_v1


### References

* OpenCV/CV2: http://askubuntu.com/questions/447409/how-to-install-opencv-2-9-for-python
* Docker Commit: http://stackoverflow.com/questions/19585028/i-lose-my-data-when-the-container-exits